<H1> WARNING!!! </H1>

This code represents my first attempt at web scraping. 
It got me banned from foodnetwork.com

<H3> Lesson 1 </H3>

Every website has its own list of rules about what is acceptable in terms of scraping, and it can be found at the extension /robots.txt, for example https://foodnetwork.com/robots.txt

Here is an introductory link if you would like to know more: https://developers.google.com/search/docs/crawling-indexing/robots/intro

<H3> Lesson 2 </H3>

It is not illegal to scrape content from a web page, but it can have the unintended consequence of overloading the target's server, so make sure you build in plenty of time.sleep() intervals to your code as an act of courtesy to your target.

#### Note: this code does not have any failsafes built in - use at your own risk or don't use at all!

In [1]:
import requests
from urllib.request import urlopen
from urllib.error import HTTPError
from urllib.error import URLError
from bs4 import BeautifulSoup
import re
import pandas as pd
import json
from collections import defaultdict
import time
import logging
import ast
import os
from collections import defaultdict
from getpass import getpass
import pymysql

The logger was to help me know when individual recipes were left out, and why.

In [2]:
logging.basicConfig(filename='./as_error.log', level=logging.DEBUG, 
                    format='%(asctime)s %(levelname)s %(name)s %(message)s')
logger=logging.getLogger(__name__)

Collect all the individual recipe pages from the "a"s

In [3]:
fna_url_list = []

for i in range(17):
    fnalink = ('https://www.foodnetwork.com/recipes/recipes-a-z/a/p/' + str(i + 1))
    fnahtml = urlopen(fnalink)
    fnabs = BeautifulSoup(fnahtml.read(), 'html.parser')
    for a in fnabs.find_all('a', {'href':re.compile('\/\/www\.foodnetwork\.com\/recipes\/.*')}):
        fna_url_list.append('https:' + a['href'])

In [4]:
fna_url_list = fna_url_list[32:-9]
# print(fna_url_list)

In [5]:
fna_clean_list = []
for i in fna_url_list:
    if (('recipes-a-z' not in i) and ('photos' not in i) and ('packages' not in i)):
        fna_clean_list.append(i)
#         print(i)

In [11]:
print(fna_clean_list[7])

https://www.foodnetwork.com/recipes/almost-famous-strawberry-lemonade-3364127


Above is an example of one of the clean recipe urls, below is the code to write the list of URLs to a text file.

In [ ]:
with open('foodnetwork_as.txt', 'w') as as_url:
    for i in fna_clean_list:
        as_url.write(str(i) + '\n')

The next three cells contain workshop code, and should note be run

In [ ]:
# lines_number = 10

# with open('foodnetwork_as.txt', 'r') as input_file:
#     head = [next(input_file) for _ in range(lines_number)]
# print(head[4])

In [ ]:
# # as_html = urlopen(fna_url_list[73])
# # as_url = fna_url_list[73]
# # as_bs = BeautifulSoup(as_html.read(), 'html.parser')
# # as_recipe_id = fna_url_list[73][-7:]
# as_html = urlopen(head[8])
# as_url = str(head[8])
# as_url = as_url.replace('\n', '')
# as_bs = BeautifulSoup(as_html.read(), 'html.parser')
# as_recipe_id = head[8][-7:]
# as_recipe_id = as_recipe_id.replace('\n', '')
# as_title = as_bs.find(class_="o-AssetTitle__a-HeadlineText").text
# as_level = as_bs.find(class_="o-RecipeInfo__m-Level").find(class_="o-RecipeInfo__a-Description").text
# as_level = 'Level: '+str(as_level)
# as_tot_time = as_bs.find(class_="o-RecipeInfo__a-Description m-RecipeInfo__a-Description--Total").text
# as_tot_time = str(as_tot_time).strip()
# as_active_time = as_bs.find(class_="o-RecipeInfo__m-Time").find(class_="o-RecipeInfo__a-Description").text
# as_active_time = str(as_active_time).strip()
# as_yield = as_bs.find(class_="o-RecipeInfo__m-Yield").find(class_="o-RecipeInfo__a-Description").text
# as_ingredients1 = []
# for i in as_bs.find_all(class_="o-Ingredients__a-Ingredient--CheckboxLabel"):
#     as_ingredients1.append(i)
# as_ingredients = []
# for i, v in enumerate(as_ingredients1):
#     as_ingredients.append(as_ingredients1[i].text)
# as_ingredients = re.sub(r'\'Deselect All\'\, ', '', str(as_ingredients))
# as_directions1 = []
# for i in as_bs.find_all(class_="o-Method__m-Step"):
#     as_directions1.append(i)
# as_directions = []
# for i, v in enumerate(as_directions1):
#     as_directions.append(as_directions1[i].text)
# as_directions = re.sub(r'\\n              ', '', str(as_directions))
# as_directions = re.sub(r'\\n            ', '', str(as_directions))
# as_directions = re.sub(r'\\xa0', '', str(as_directions))
# as_image = as_bs.find(class_="m-MediaBlock__m-MediaWrap")
# as_image = re.sub(r'<div class="m-MediaBlock__m-MediaWrap">\n', '', str(as_image))
# as_image = re.sub(r'(.*?)src=\"', '', str(as_image))
# as_image = re.sub(r'\" srcset=+(.*?)width+(.*?)\>', '', str(as_image))
# as_image = str(as_image)[:-7]
# as_image = 'https:'+str(as_image)
# # as_image = 'https:'+as_image
# as_submitter = as_bs.find(class_="o-Attribution__a-Name").text
# as_submitter = re.sub(r'(.*?)Recipe courtesy of ', '', str(as_submitter))
# as_submitter = as_submitter.replace('\n', '')
# as_submitter = as_submitter.strip()
# as_num_reviews = as_bs.find(class_="reviews-ct")
# as_num_stars = as_bs.find(class_="sr-only")
# as_categories = 'NA'
# as_nut_info_head = as_bs.find(class_="m-NutritionTable__a-Headline")
# as_nut_info_cont = as_bs.find(class_="m-NutritionTable__a-Description")

In [ ]:
# as_nut_info = as_bs.find_all(id_="nutrition-content")
# btn_onclick_list = [a.get('onclick') for a in as_bs.find_all('button')]
# for click in btn_onlclick_list:
#      a = re.findall('class_="m-NutritionTable__a-Headline"', click)[0]
#      print(a)
# print(btn_onclick_list)
# print(as_nut_info)

Create a dictionary that will populate with individual recipe contents as each recipe is scraped

In [6]:
as_dict = defaultdict(list)

In [7]:
def fna_scrape(input_file):
    as_html = urlopen(input_file)
    as_url = str(input_file)
    as_url = as_url.replace('\n', '')
    as_bs = BeautifulSoup(as_html.read(), 'html.parser')
    as_recipe_id = str(input_file)[-7:]
    as_recipe_id = as_recipe_id.replace('\n', '')
    as_title = as_bs.find(class_="o-AssetTitle__a-HeadlineText").text
    as_level = as_bs.find(class_="o-RecipeInfo__m-Level").find(class_="o-RecipeInfo__a-Description").text
    as_level = 'Level: '+str(as_level)
    as_tot_time = as_bs.find(class_="o-RecipeInfo__a-Description m-RecipeInfo__a-Description--Total").text
    as_tot_time = str(as_tot_time).strip()
    as_active_time = as_bs.find(class_="o-RecipeInfo__m-Time").find(class_="o-RecipeInfo__a-Description").text
    as_active_time = str(as_active_time).strip()
    as_yield = as_bs.find(class_="o-RecipeInfo__m-Yield").find(class_="o-RecipeInfo__a-Description").text
    as_ingredients1 = []
    for i in as_bs.find_all(class_="o-Ingredients__a-Ingredient--CheckboxLabel"):
        as_ingredients1.append(i)
    as_ingredients = []
    for i, v in enumerate(as_ingredients1):
        as_ingredients.append(as_ingredients1[i].text)
    as_ingredients = re.sub(r'\'Deselect All\'\, ', '', str(as_ingredients))
    as_directions1 = []
    for i in as_bs.find_all(class_="o-Method__m-Step"):
        as_directions1.append(i)
    as_directions = []
    for i, v in enumerate(as_directions1):
        as_directions.append(as_directions1[i].text)
    as_directions = re.sub(r'\\n              ', '', str(as_directions))
    as_directions = re.sub(r'\\n            ', '', str(as_directions))
    as_directions = re.sub(r'\\xa0', '', str(as_directions))
    as_image = as_bs.find(class_="m-MediaBlock__m-MediaWrap")
    as_image = re.sub(r'<div class="m-MediaBlock__m-MediaWrap">\n', '', str(as_image))
    as_image = re.sub(r'(.*?)src=\"', '', str(as_image))
    as_image = re.sub(r'\" srcset=+(.*?)width+(.*?)\>', '', str(as_image))
    as_image = str(as_image)[:-7]
    as_image = 'https:'+str(as_image)
    as_submitter = as_bs.find(class_="o-Attribution__a-Name").text
    as_submitter = re.sub(r'(.*?)Recipe courtesy of ', '', str(as_submitter))
    as_submitter = as_submitter.replace('\n', '')
    as_submitter = as_submitter.strip()
    as_num_reviews = as_bs.find(class_="reviews-ct")
    as_num_stars = as_bs.find(class_="sr-only")
    as_categories = 'NA'
    as_dict['RECIPE_ID'].append(as_recipe_id)
    as_dict['url'].append(as_url)
    as_dict['TITLE'].append(as_title)
    as_dict['IMAGE'].append(as_image)
    as_dict['INGREDIENTS'].append(str(as_ingredients))
    as_dict['DIRECTIONS'].append(str(as_directions))
    as_dict['SUBMITTER'].append(as_submitter)
    as_dict['DESCRIPTION'].append(as_level)
    as_dict['CATEGORIES'].append(as_categories)
    as_dict['NUM_RATINGS'].append(as_num_reviews)
    as_dict['NUM_STARS'].append(as_num_stars)
    as_dict['PREP_TIME'].append(as_active_time)
    as_dict['READY_IN'].append(as_tot_time)
    as_dict['NUM_SERVINGS'].append(as_yield)

Scrape the recipes and manage exceptions as they arise so they don't interrupt the scraping session.

In [8]:
for i in range(len(fna_clean_list)):
    try:
        fna_scrape(fna_clean_list[i])
    except AttributeError as e:
        logger.error(e)
        print(i, e)
    except HTTPError as e2:
        logger.error(e2)
        print(i, e2)
    except URLError as e3:
        logger.error(e3)
        print(i, e3)
    time.sleep(0.25)

5 'NoneType' object has no attribute 'text'
7 'NoneType' object has no attribute 'text'
14 'NoneType' object has no attribute 'find'
24 'NoneType' object has no attribute 'text'
28 'NoneType' object has no attribute 'find'
32 'NoneType' object has no attribute 'text'
35 'NoneType' object has no attribute 'text'
38 'NoneType' object has no attribute 'text'
41 'NoneType' object has no attribute 'find'
45 'NoneType' object has no attribute 'find'
50 'NoneType' object has no attribute 'find'
54 'NoneType' object has no attribute 'find'
60 'NoneType' object has no attribute 'find'
70 'NoneType' object has no attribute 'find'
81 'NoneType' object has no attribute 'find'
83 'NoneType' object has no attribute 'find'
91 'NoneType' object has no attribute 'text'
92 'NoneType' object has no attribute 'text'
98 'NoneType' object has no attribute 'text'
99 'NoneType' object has no attribute 'text'
100 'NoneType' object has no attribute 'find'
110 'NoneType' object has no attribute 'text'
112 'NoneT

1294 'NoneType' object has no attribute 'find'
1295 'NoneType' object has no attribute 'find'
1296 'NoneType' object has no attribute 'text'
1299 'NoneType' object has no attribute 'find'
1301 'NoneType' object has no attribute 'find'
1335 'NoneType' object has no attribute 'text'
1351 'NoneType' object has no attribute 'text'
1362 'NoneType' object has no attribute 'text'
1370 'NoneType' object has no attribute 'text'
1383 'NoneType' object has no attribute 'find'
1398 'NoneType' object has no attribute 'text'
1406 'NoneType' object has no attribute 'find'
1407 'NoneType' object has no attribute 'text'
1411 'NoneType' object has no attribute 'find'
1439 'NoneType' object has no attribute 'text'
1447 'NoneType' object has no attribute 'find'
1476 'NoneType' object has no attribute 'text'
1487 'NoneType' object has no attribute 'find'
1488 'NoneType' object has no attribute 'text'
1491 'NoneType' object has no attribute 'find'
1497 'NoneType' object has no attribute 'text'
1500 'NoneTyp

The next four cells contain workshop code that should not be run.

In [ ]:
# as_dict = defaultdict(list)

In [ ]:
# as_dict['RECIPE_ID'].append(as_recipe_id)
# as_dict['url'].append(as_url)
# as_dict['TITLE'].append(as_title)
# as_dict['IMAGE'].append(as_image)
# as_dict['INGREDIENTS'].append(str(as_ingredients))
# as_dict['DIRECTIONS'].append(str(as_directions))
# as_dict['SUBMITTER'].append(as_submitter)
# as_dict['DESCRIPTION'].append(as_level)
# as_dict['CATEGORIES'].append(as_categories)
# as_dict['NUM_RATINGS'].append(as_num_reviews)
# as_dict['NUM_STARS'].append(as_num_stars)
# as_dict['PREP_TIME'].append(as_active_time)
# as_dict['READY_IN'].append(as_tot_time)
# as_dict['NUM_SERVINGS'].append(as_yield)

In [ ]:
# as_dict = {'RECIPE_ID': as_recipe_id,
#            'URL': as_url,
#            'TITLE': as_title,
#            'IMAGE': 'https:'+str(as_image),
#            'INGREDIENTS': str(as_ingredients),
#            'DIRECTIONS': str(as_directions),
#            'SUBMITTER': as_submitter,
#            'DESCRIPTION': as_level,
#            'CATEGORIES': as_categories,
#            'NUM_RATINGS': as_num_reviews,
#            'NUM_STARS': as_num_stars,
#            'PREP_TIME': as_active_time,
#            'READY_IN': as_tot_time,
#            'NUM_SERVINGS': as_yield
#            }

# as_dict['INGREDIENTS'] = re.sub(r'\'Deselect All\'\, ', '', as_dict['INGREDIENTS'])

# as_dict['DIRECTIONS'] = re.sub(r'\\n              ', '', as_dict['DIRECTIONS'])
# as_dict['DIRECTIONS'] = re.sub(r'\\n            ', '', as_dict['DIRECTIONS'])
# as_dict['DIRECTIONS'] = re.sub(r'\\xa0', '', as_dict['DIRECTIONS'])
# 
# print(as_dict)

In [ ]:
# as_df = pd.DataFrame(as_dict.items())
# display(as_df)

Convert the dictionary to pandas dataframe, then remove duplicates

In [9]:
as_df2 = pd.DataFrame.from_dict(as_dict)
# display(as_df2)
as_df2.shape

(2149, 14)

In [10]:
as_df3 = as_df2.drop_duplicates(keep='first')
as_df3.shape

(2081, 14)

Create the table in your database (only run once)

In [ ]:
create_recipes_test3_table_query = """
CREATE TABLE recipes_test3(
    RECIPE_ID VARCHAR(48),
    URL VARCHAR(260),
    TITLE VARCHAR(400),
    IMAGE VARCHAR(200),
    INGREDIENTS VARCHAR(2000),
    DIRECTIONS VARCHAR(5000),
    SUBMITTER VARCHAR(100),
    DESCRIPTION VARCHAR(810),
    CATEGORIES VARCHAR(100),
    NUM_RATINGS VARCHAR(100),
    NUM_STARS VARCHAR(100),
    PREP_TIME VARCHAR(2220),
    READY_IN VARCHAR(100),
    NUM_SERVINGS VARCHAR(100)
)
"""

with pymysql.connect(
    user=input("Enter username: "),
    password=getpass("Enter password: "),
    host="localhost",
    db="recipe_test",
    ) as connection:
        with connection.cursor() as cursor:
            cursor.execute(create_recipes_test3_table_query)
            connection.commit()

There were outliers among the recipes that demanded more characters than I had anticipated when I created the table, leading to errors when I exported the pandas df to the table. The following cell gets the maximum character length in a df field, which I used to ALTER the table - manually, in MySQL, using SQL code not here using Python.

In [ ]:
print(as_df2.IMAGE.str.len().max())

Export pandas df to MySQL database table. The last two cells repeat the process, pushing to the remote Azure db (obvious;y I have left out the details and credentials)

In [ ]:
with pymysql.connect(
    host="localhost",
    user=input("Enter username: "),
    password=getpass("Enter password: "),
    db="recipe_test"
) as connection:
    with connection.cursor() as cursor:
        for i,row in as_df2.iterrows():
            sql = "INSERT INTO recipe_test.recipes_test3 (\
            RECIPE_ID, URL, TITLE, IMAGE, INGREDIENTS, DIRECTIONS,\
            SUBMITTER, DESCRIPTION, CATEGORIES, NUM_RATINGS, NUM_STARS,\
            PREP_TIME, READY_IN, NUM_SERVINGS) VALUES (" + "%s,"*(len(row)-1) + "%s)"
            cursor.execute(sql, tuple(row))
            connection.commit()

In [ ]:
create_recipes_new_test_table_query = """
CREATE TABLE recipes_new_test_fn(
    RECIPE_ID VARCHAR(48),
    URL VARCHAR(260),
    TITLE VARCHAR(400),
    IMAGE VARCHAR(1500),
    INGREDIENTS VARCHAR(2200),
    DIRECTIONS VARCHAR(6700),
    SUBMITTER VARCHAR(220),
    DESCRIPTION VARCHAR(810),
    CATEGORIES VARCHAR(100),
    NUM_RATINGS VARCHAR(100),
    NUM_STARS VARCHAR(100),
    PREP_TIME VARCHAR(2220),
    READY_IN VARCHAR(100),
    NUM_SERVINGS VARCHAR(100)
)
"""

with pymysql.connect(
    user=input("Enter username: "),
    password=getpass("Enter password: "),
    host=,
    port=,
    db="recipator_db",
    ssl_ca="",
    ssl_disabled=False
    ) as connection:
        with connection.cursor() as cursor:
            cursor.execute(create_recipes_new_test_table_query)
            connection.commit()

In [ ]:
with pymysql.connect(
    user=input("Enter username: "),
    password=getpass("Enter password: "),
    host=,
    port=,
    db="recipator_db",
    ssl_ca=,
    ssl_disabled=False
    ) as connection:
        with connection.cursor() as cursor:
            for i,row in as_df2.iterrows():
                sql = "INSERT INTO recipator_db.recipes_new_test_fn (\
                RECIPE_ID, URL, TITLE, IMAGE, INGREDIENTS, DIRECTIONS,\
                SUBMITTER, DESCRIPTION, CATEGORIES, NUM_RATINGS, NUM_STARS,\
                PREP_TIME, READY_IN, NUM_SERVINGS) VALUES (" + "%s,"*(len(row)-1) + "%s)"
                cursor.execute(sql, tuple(row))
                connection.commit()  